<a href="https://colab.research.google.com/github/Karan1106/Assignment3/blob/master/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
import random
from scipy import ndarray

BUILDING CONVOLUTIONAL NEURAL NETWORK

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras import backend as K

Using TensorFlow backend.


In [5]:
# dimensions of our images.
#input_shape=(256,256,3)
img_width, img_height = 150, 150


trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/gdrive/My Drive/Colab Notebooks/A4 Augmented images")
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/gdrive/My Drive/Colab Notebooks/Scrap_test")

Found 1842 images belonging to 3 classes.
Found 308 images belonging to 3 classes.


In [6]:

nb_train_samples=1842
nb_validation_samples=308
batch_size=32
epochs=20

In [7]:
#model building
model = Sequential()
model.add(Conv2D(input_shape=(256,256,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

#flatten layer
model.add(Flatten()) # Output convert into one dimension layer and will go to Dense layer
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Dense(3, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [8]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)    

SGD OPTIMIZER


In [9]:
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("Modelsgd.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = SGD(lr=0.01, momentum=0.5, decay=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [10]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])


Epoch 1/20
57/57 [==============================] - 408s 7s/step - loss: 0.9017 - accuracy: 0.5431 - val_loss: 0.9035 - val_accuracy: 0.4861

Epoch 00001: val_accuracy improved from -inf to 0.48611, saving model to Modelsgd.h5
Epoch 2/20
57/57 [==============================] - 76s 1s/step - loss: 0.5374 - accuracy: 0.6630 - val_loss: 0.9362 - val_accuracy: 0.5543

Epoch 00002: val_accuracy improved from 0.48611 to 0.55435, saving model to Modelsgd.h5
Epoch 3/20
57/57 [==============================] - 66s 1s/step - loss: 0.5012 - accuracy: 0.6765 - val_loss: 0.5118 - val_accuracy: 0.6304

Epoch 00003: val_accuracy improved from 0.55435 to 0.63043, saving model to Modelsgd.h5
Epoch 4/20
57/57 [==============================] - 66s 1s/step - loss: 0.4816 - accuracy: 0.6876 - val_loss: 0.4280 - val_accuracy: 0.6884

Epoch 00004: val_accuracy improved from 0.63043 to 0.68841, saving model to Modelsgd.h5
Epoch 5/20
57/57 [==============================] - 66s 1s/step - loss: 0.4705 - accu

ADAM

In [11]:
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint= ModelCheckpoint("Modeladam.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = Adam(lr=0.01, decay=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 69s 1s/step - loss: 4.4517 - accuracy: 0.3746 - val_loss: 75.9680 - val_accuracy: 0.3507

Epoch 00001: val_accuracy improved from -inf to 0.35069, saving model to Modeladam.h5
Epoch 2/20
57/57 [==============================] - 66s 1s/step - loss: 1.0073 - accuracy: 0.3746 - val_loss: 4.7033 - val_accuracy: 0.4058

Epoch 00002: val_accuracy improved from 0.35069 to 0.40580, saving model to Modeladam.h5
Epoch 3/20
57/57 [==============================] - 66s 1s/step - loss: 0.9694 - accuracy: 0.4354 - val_loss: 0.8944 - val_accuracy: 0.4819

Epoch 00003: val_accuracy improved from 0.40580 to 0.48188, saving model to Modeladam.h5
Epoch 4/20
57/57 [==============================] - 66s 1s/step - loss: 0.9188 - accuracy: 0.4564 - val_loss: 0.6570 - val_accuracy: 0.5725

Epoch 00004: val_accuracy improved from 0.48188 to 0.57246, saving model to Modeladam.h5
Epoch 5/20
57/57 [==============================] - 66s 1s/step - loss: 0.8954 - a

NADAM

In [13]:
from keras.callbacks import Callback
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint= ModelCheckpoint("Modelnadam.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [14]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 69s 1s/step - loss: 0.7121 - accuracy: 0.5785 - val_loss: 1.0655 - val_accuracy: 0.4549

Epoch 00001: val_accuracy improved from -inf to 0.45486, saving model to Modelnadam.h5
Epoch 2/20
57/57 [==============================] - 67s 1s/step - loss: 0.6654 - accuracy: 0.5884 - val_loss: 1.1781 - val_accuracy: 0.3804

Epoch 00002: val_accuracy did not improve from 0.45486
Epoch 3/20
57/57 [==============================] - 67s 1s/step - loss: 0.6214 - accuracy: 0.6144 - val_loss: 0.5089 - val_accuracy: 0.6703

Epoch 00003: val_accuracy improved from 0.45486 to 0.67029, saving model to Modelnadam.h5
Epoch 4/20
57/57 [==============================] - 67s 1s/step - loss: 0.5724 - accuracy: 0.6122 - val_loss: 0.6200 - val_accuracy: 0.6594

Epoch 00004: val_accuracy did not improve from 0.67029
Epoch 5/20
57/57 [==============================] - 67s 1s/step - loss: 0.5283 - accuracy: 0.6575 - val_loss: 0.4800 - val_accuracy: 0.6341

Epoch 00

RMSprop

In [15]:
from keras.callbacks import Callback
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("Modelrmsprop.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
momentums = [0.0, 0.5, 0.9, 0.99]
opt = RMSprop(lr=0.01)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 68s 1s/step - loss: 1.7592 - accuracy: 0.3580 - val_loss: 1.1373 - val_accuracy: 0.3507

Epoch 00001: val_accuracy improved from -inf to 0.35069, saving model to Modelrmsprop.h5
Epoch 2/20
57/57 [==============================] - 66s 1s/step - loss: 0.9797 - accuracy: 0.4829 - val_loss: 0.4205 - val_accuracy: 0.6739

Epoch 00002: val_accuracy improved from 0.35069 to 0.67391, saving model to Modelrmsprop.h5
Epoch 3/20
57/57 [==============================] - 66s 1s/step - loss: 0.7425 - accuracy: 0.6195 - val_loss: 1.1811 - val_accuracy: 0.6594

Epoch 00003: val_accuracy did not improve from 0.67391
Epoch 4/20
57/57 [==============================] - 65s 1s/step - loss: 0.6589 - accuracy: 0.6565 - val_loss: 4.7983 - val_accuracy: 0.4565

Epoch 00004: val_accuracy did not improve from 0.67391
Epoch 5/20
57/57 [==============================] - 65s 1s/step - loss: 0.6269 - accuracy: 0.6442 - val_loss: 0.4429 - val_accuracy: 0.6703

Epoc

 Nesterov Accelerated Gradient Descent



In [17]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD

checkpoint= ModelCheckpoint("ModelNag.h5", monitor="val_accuracy",mode='max', save_best_only=True,verbose=1)

earlystop= EarlyStopping(monitor="val_accuracy",min_delta=0,patience=5,verbose=1,restore_best_weights=True)

reduce_lr= ReduceLROnPlateau(monitor="val_accuracy", factor=0.2, patience=5, verbose=1, min_delta=0.001)

#putiing callbacks into callback list
callbacks = [earlystop, checkpoint, reduce_lr]
opt = SGD(lr=0.01, nesterov=True)
#we use a very small learning rate
model.compile(loss = 'categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [18]:
history=model.fit_generator(
    traindata,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=testdata,
    validation_steps=nb_validation_samples//batch_size,
    callbacks = [earlystop, checkpoint, reduce_lr])

Epoch 1/20
57/57 [==============================] - 67s 1s/step - loss: 0.7542 - accuracy: 0.5967 - val_loss: 2.2153 - val_accuracy: 0.3299

Epoch 00001: val_accuracy improved from -inf to 0.32986, saving model to ModelNag.h5
Epoch 2/20
57/57 [==============================] - 65s 1s/step - loss: 0.7287 - accuracy: 0.6210 - val_loss: 2.6945 - val_accuracy: 0.3333

Epoch 00002: val_accuracy improved from 0.32986 to 0.33333, saving model to ModelNag.h5
Epoch 3/20
57/57 [==============================] - 65s 1s/step - loss: 0.7250 - accuracy: 0.6061 - val_loss: 1.5040 - val_accuracy: 0.3370

Epoch 00003: val_accuracy improved from 0.33333 to 0.33696, saving model to ModelNag.h5
Epoch 4/20
57/57 [==============================] - 65s 1s/step - loss: 0.7402 - accuracy: 0.6127 - val_loss: 1.6601 - val_accuracy: 0.3188

Epoch 00004: val_accuracy did not improve from 0.33696
Epoch 5/20
57/57 [==============================] - 65s 1s/step - loss: 0.7015 - accuracy: 0.6116 - val_loss: 2.0346 - v